## Mithilfe von PyMuPDF pdf-Dateien auslesen: Die Linke



In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.1 MB/s eta 0:00:00


In [2]:
## pdf-Datei anschauen: Die ersten 3 Seiten enthalten Deckblatt + Inhaltsverzeichnis - das kann also übersprungen werden

# Als erstes die Formatinformation extrahieren: Also Text mit Schriftgröße und Dicke der Schrift, um später die Überschriften zu finden und
# zur Gliederung verwenden zu können

# Text ab vierter Seite extrahieren inkl. Formatinfos:

import pymupdf as fitz

def extract_text_with_formatting(pdf_path, skip_pages=3):
    doc = fitz.open(pdf_path)
    text_data = []

    for page_num in range(skip_pages, len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Speichere Text + Schriftgröße + Fettdruck-Info
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0  # Prüft, ob der Text fett ist
                        })

    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_Die_Linke.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# ersten paar Textabschnitte mit Formatierung anzeigen
for item in text_data[:1000]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}")


Text: Alle wollen regieren.  , Size: 32.0, Bold: False
Text: Wir wollen verändern., Size: 32.0, Bold: False
Text: Reichtum teilen. Preise senken. , Size: 28.0, Bold: False
Text: Füreinander., Size: 28.0, Bold: False
Text: Liebe Wähler*innen,, Size: 15.0, Bold: False
Text: wir sind verschieden, aber die meisten von uns wollen das , Size: 9.0, Bold: False
Text: Gleiche: eine Gesellschaft, in der man sich gegenseitig , Size: 9.0, Bold: False
Text: zuhört und unterstützt, in der alle gesehen und gehört , Size: 9.0, Bold: False
Text: werden. Ein planbares Leben mit einer sicheren Zukunft, , Size: 9.0, Bold: False
Text: die Platz lässt für eigene Ziele und Entscheidungen. Eine , Size: 9.0, Bold: False
Text: Politik, die Verantwortung übernimmt, an der Seite der , Size: 9.0, Bold: False
Text: Menschen steht und sich um ihre Probleme kümmert., Size: 9.0, Bold: False
Text: Wir, Die Linke, kämpfen dafür, wir sind überzeugt: Es ist , Size: 9.0, Bold: False
Text: möglich. Ein Leben, in dem alle ei

In [3]:
## Output anschauen: Normale Schrift ist in Schriftgröße 9, Überschriften sind größer, Seitenzahlen stehen in einzelnen Zeilen

import pandas as pd


def split_sections(text_data, min_size=9, bold_threshold=True):
    sections = []
    current_section = []

    for item in text_data:
        text = item["text"].strip() # führende und nachvollgende Leerzeichen entfernen (da Seitenzahlen sonst nicht immer erkannt werden)

        if item["size"] > min_size or item["bold"] == bold_threshold: # Überschriften finden und Abschnitte definieren
            if current_section:
                sections.append(" ".join(current_section).strip())
            current_section = []  # Abschnitt zurücksetzen

        elif text.isdigit() and len(text) <= 4: # Seitenzahlen überspringen
            continue
        else:
            current_section.append(text)

    if current_section: # Abschnitt hinzufügen
        sections.append(" ".join(current_section).strip())

    return sections


# Abschnitte extrahieren
sections = split_sections(text_data)

# Erstelle eine Tabelle mit den Abschnitten
df = pd.DataFrame(sections, columns=["Abschnitt"])

print(df[:10])


                                           Abschnitt
0  wir sind verschieden, aber die meisten von uns...
1  Drei Jahre Ampel-Ausfall haben Deutschland noc...
2  Sparen beginnt für viele Menschen inzwischen b...
3  Alle brauchen bezahlbare und stabile Energiepr...
4  Um Preisschocks in Zukunft zu verhindern, brau...
5  Bezahlbares Wohnen ist die zentrale soziale Fr...
6  Mieten müssen gesenkt und wirksam reguliert we...
7  Wir wollen 20 Milliarden Euro im Jahr in gemei...
8  Auch wenn Neubau allein die Miete nicht senkt,...
9  Leerstand ist ein unterschätztes Problem. Deut...


In [4]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/die_linke.csv", index=False, encoding="utf-8")